In [1]:
import pandas as pd
import numpy as np

In [2]:
diag = pd.read_csv('mimic-iii-clinical-database-demo-1.4/DIAGNOSES_ICD.csv')
lab = pd.read_csv('mimic-iii-clinical-database-demo-1.4/LABEVENTS.csv')

In [3]:
unique_values = diag['hadm_id'].dropna().unique()
num_hadm_id = len(unique_values)
hadm_id_index = dict()
for i, value in enumerate(unique_values):
    hadm_id_index[value] = i

In [4]:
hadm_id_subject_id = diag.groupby('subject_id')['hadm_id'].unique()

In [5]:
unique_values = diag['icd9_code'].dropna().unique()
num_icd9_code = len(unique_values)
icd9_code_index = dict()
for i, value in enumerate(unique_values):
    icd9_code_index[value] = i

In [6]:
unique_values = lab['itemid'].dropna().unique()
num_lab_item = len(unique_values)
lab_item_index = dict()
for i, value in enumerate(unique_values):
    lab_item_index[value] = i

In [8]:
diag_data = np.zeros([num_hadm_id, num_icd9_code], dtype=bool)
lab_data = np.zeros([num_hadm_id, num_lab_item], dtype=bool)
# for continuous lab event node
# lab_data = np.ones([num_hadm_id, num_lab_item]) * np.nan

for i in range(len(diag)):
    hadm_id = diag.loc[i]['hadm_id']
    icd9_code = diag.loc[i]['icd9_code']
    diag_data[hadm_id_index[hadm_id], icd9_code_index[icd9_code]] = True

for i in range(len(lab)):
    subject_id = lab.loc[i]['subject_id']
    hadm_id = lab.loc[i]['hadm_id']
    itemid = lab.loc[i]['itemid']
    # valuenum = lab.loc[i]['valuenum']
    if np.isnan(hadm_id):
        if len(hadm_id_subject_id[subject_id]) != 1:
            continue
        else:
            hadm_id = hadm_id_subject_id[subject_id][0]
    lab_data[hadm_id_index[hadm_id], lab_item_index[itemid]] = (lab.loc[i]['flag']=='abnormal')
    # lab_data[hadm_id_index[hadm_id], lab_item_index[itemid]] = valuenum

In [9]:
diag_col_name = []
for icd9_code in icd9_code_index:
    diag_col_name.append('icd9_code_{}'.format(icd9_code))

lab_col_name = []
for itemid in lab_item_index:
    lab_col_name.append('lab_itemid_{}'.format(icd9_code))

diag_df = pd.DataFrame(diag_data, columns=diag_col_name)
lab_df = pd.DataFrame(lab_data, columns=lab_col_name)

df = pd.concat([diag_df, lab_df], axis=1)
df.to_csv('mimic-iii.csv')